In [63]:
from pathlib import Path
import numpy as np
import copy

In [64]:
def _load_from_npz(path_npz: str | Path):
    """Recria a lista de arrays salva via np.savez/np.savez_compressed."""
    with np.load(path_npz, allow_pickle=False) as data:
        keys = list(data.files)
        # mantém a ordem a0, a1, ... se existir
        if all(k.startswith("a") and k[1:].isdigit() for k in keys):
            keys.sort(key=lambda k: int(k[1:]))
        else:
            keys.sort()
        return [data[k] for k in keys]

def build_U_clients(root_dir: str = "similarity"):
    """Carrega cada '<...>_<cid>.npz' de root_dir e monta U_clients."""
    root = Path(root_dir)

    def _sort_key(p: Path):
        stem = p.stem
        try:
            # ordena por (aggregation, selection, participants, model, cid)
            a, s, pt, m, cid = stem.rsplit("_", 4)
            return (a, s, pt, m, int(cid))
        except ValueError:
            return (stem, "", "", "", 0)

    U_clients = []
    for p in sorted(root.glob("*.npz"), key=_sort_key):
        U_temp = _load_from_npz(p)              # lista de ndarrays
        U_clients.append(copy.deepcopy(np.hstack(U_temp)))  # <- sua linha
    return U_clients


def calculating_adjacency(clients_idxs, U): 
        
    nclients = len(clients_idxs)
    
    sim_mat = np.zeros([nclients, nclients])
    for idx1 in range(nclients):
        for idx2 in range(nclients):
            #print(idx1)
            #print(U)
            #print(idx1)
            U1 = copy.deepcopy(U[clients_idxs[idx1]])
            U2 = copy.deepcopy(U[clients_idxs[idx2]])
            
            #sim_mat[idx1,idx2] = np.where(np.abs(U1.T@U2) > 1e-2)[0].shape[0]
            #sim_mat[idx1,idx2] = 10*np.linalg.norm(U1.T@U2 - np.eye(15), ord='fro')
            #sim_mat[idx1,idx2] = 100/np.pi*(np.sort(np.arccos(U1.T@U2).reshape(-1))[0:4]).sum()
            mul = np.clip(U1.T@U2 ,a_min =-1.0, a_max=1.0)
            sim_mat[idx1,idx2] = np.min(np.arccos(mul))*180/np.pi
           
    return sim_mat

def to_similarity_matrix(D: np.ndarray, d_max: float | None = None) -> np.ndarray:
    """
    Converte uma matriz de distâncias D (quadrada, simétrica) em matriz de similaridade S
    usando S_ij = 1 - D_ij / d_max. Por padrão, d_max é o maior valor fora da diagonal.
    A diagonal da saída é fixada em 1.0. Retorna S no intervalo [0, 1] (clamp numérico).

    Parâmetros
    ----------
    D : np.ndarray
        Matriz de distâncias (n x n), simétrica.
    d_max : float | None
        Máxima distância para normalização. Se None, usa o máximo off-diagonal de D.

    Retorno
    -------
    np.ndarray
        Matriz de similaridade (n x n), com diagonal = 1.0.
    """
    if D.ndim != 2 or D.shape[0] != D.shape[1]:
        raise ValueError("D deve ser uma matriz quadrada.")
    if not np.isfinite(D).all():
        raise ValueError("D contém valores não finitos.")
    if (D < 0).any():
        raise ValueError("D contém distâncias negativas.")

    n = D.shape[0]
    offdiag_mask = ~np.eye(n, dtype=bool)
    if d_max is None:
        if offdiag_mask.sum() == 0:
            raise ValueError("Matriz 1x1 não possui elementos fora da diagonal.")
        d_max = float(D[offdiag_mask].max())

    S = np.ones_like(D, dtype=float)

    if d_max > 0:
        S = 1.0 - (D.astype(float) / d_max)
        # Ajustes numéricos e diagonal
        S = np.clip(S, 0.0, 1.0)
        np.fill_diagonal(S, 1.0)
    else:
        # Caso degenerado: todas as off-diagonais são zero -> similaridade 1 fora da diagonal
        S = np.ones_like(D, dtype=float)
        np.fill_diagonal(S, 1.0)

    return S


def mean_similarity(S: np.ndarray) -> float:
    """
    Calcula a similaridade média de uma matriz de similaridade S (simétrica),
    ignorando a diagonal (i.e., média das entradas i<j).

    Parâmetros
    ----------
    S : np.ndarray
        Matriz de similaridade (n x n), simétrica.

    Retorno
    -------
    float
        Similaridade média fora da diagonal.
    """
    if S.ndim != 2 or S.shape[0] != S.shape[1]:
        raise ValueError("S deve ser uma matriz quadrada.")
    n = S.shape[0]
    if n < 2:
        raise ValueError("Matriz 1x1 não possui pares fora da diagonal.")

    iu = np.triu_indices(n, k=1)
    return float(S[iu].mean())


def to_similarity_matrix_angle_cos(D: np.ndarray) -> np.ndarray:
    """
    Converte uma matriz de ângulos (radianos) em matriz de similaridade:
        S = cos(D)
    A saída fica em [0,1] se D ∈ [0, pi/2]. A diagonal é forçada para 1.0.

    Parâmetros
    ----------
    D : np.ndarray
        Matriz (n x n) de ângulos em radianos, simétrica.

    Retorno
    -------
    np.ndarray
        Matriz de similaridade (n x n).
    """
    if D.ndim != 2 or D.shape[0] != D.shape[1]:
        raise ValueError("D deve ser uma matriz quadrada.")
    if not np.isfinite(D).all():
        raise ValueError("D contém valores não finitos.")
    if (D < 0).any():
        raise ValueError("D contém ângulos negativos (esperado: [0, pi/2]).")
    
    D_rad = np.deg2rad(D)
    S = np.cos(D_rad.astype(float))
    S = np.clip(S, 0.0, 1.0)   # segurança numérica
    np.fill_diagonal(S, 1.0)
    return S


def mean_similarity(S: np.ndarray) -> float:
    """
    Similaridade média fora da diagonal (matriz simétrica).

    Parâmetros
    ----------
    S : np.ndarray
        Matriz de similaridade (n x n), simétrica.

    Retorno
    -------
    float
        Média das entradas i<j (exclui diagonal).
    """
    if S.ndim != 2 or S.shape[0] != S.shape[1]:
        raise ValueError("S deve ser uma matriz quadrada.")
    n = S.shape[0]
    if n < 2:
        raise ValueError("Matriz 1x1 não possui pares fora da diagonal.")
    iu = np.triu_indices(n, k=1)
    return float(S[iu].mean())


In [65]:
U_clients = build_U_clients("../similarity")

In [66]:
U_clients

[array([[ 0.00601246,  0.01540829, -0.00308162,  0.00397951, -0.0248435 ,
          0.0122429 ],
        [ 0.00462666,  0.01640954, -0.00370602,  0.004454  , -0.01988724,
         -0.00732829],
        [-0.00091657,  0.0189298 , -0.00297306, -0.00025277, -0.01078389,
          0.00375773],
        ...,
        [-0.02011306,  0.0222451 ,  0.01396489, -0.02001888, -0.02027091,
          0.02172138],
        [-0.02037476,  0.02172361,  0.01036269, -0.02355901, -0.0198682 ,
          0.02088626],
        [-0.02115984,  0.02446212,  0.00846366, -0.03263182, -0.01432133,
          0.00011973]], shape=(3072, 6)),
 array([[ 1.77288056e-02, -1.67801336e-02,  3.39257450e-02, ...,
          1.00476101e-02,  4.31273578e-03, -8.12082615e-03],
        [ 1.73931782e-02, -1.76197708e-02,  3.52961479e-02, ...,
          1.00991861e-02,  5.50228994e-03, -1.10038332e-02],
        [ 1.76245910e-02, -1.73695611e-02,  3.55919292e-02, ...,
          9.93534309e-03,  4.80470570e-03, -1.11725210e-02],
        

In [67]:
clients_idxs = np.arange(100)
dist_mat = calculating_adjacency(clients_idxs, U_clients)

In [68]:
sim_mat = to_similarity_matrix_angle_cos(dist_mat)

In [69]:
mean_similarity(sim_mat)

0.9205224284492979

In [70]:
# S = to_similarity_matrix(dist_mat)        # normaliza por max off-diagonal (aqui, 4.0)
# s_mean = mean_similarity(S)        # média fora da diagonal
# print(S)
# print(s_mean)